In [4]:
import requests
import xml.etree.ElementTree as ET
from string import digits
from string import punctuation
import string
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('danish')
regex = re.compile('[^a-zA-Z]')

In [5]:
from requests_futures.sessions import FuturesSession
session = FuturesSession(max_workers=10)

In [14]:
pages = range(0, 100)
years = range(1925, 1983, 1)
months = range(1,13)
month= 6

In [ ]:
data = {}
for year in years:
    ids = []
    year_month_start = "{0}-{1:02d}".format(year, 1)
    print(year_month_start)
    year_month_end = "{0}-{1:02d}".format(year+1, 12)
    for page in pages:
        url ="http://api.larm.fm/v6/View/Get?view=Search&sort=PubStartDate%2Bdesc&filter=%28PubStartDate:[{}-01T00:00:00Z%20TO%20{}-30T00:00:00Z]%20AND%20%28Type%3ASchedule%20OR%20Type%3AScheduleNote%29%29&pageIndex={}&pageSize=20&sessionGUID=049da351-b81f-424e-82c4-1162926d3688&format=json&userHTTPStatusCodes=False".format(year_month_start, year_month_end,page)
        r = requests.get(url)
        if r.json()["ModuleResults"][0]["Count"] is 0:
            break
        else:
            ids_ = [i['Id'] for i in  r.json()["ModuleResults"][0]["Results"]]
            for id in ids_:
                ids.append(id)
    data[year_month_start] = ids

1925-01
1926-01
1927-01
1928-01
1929-01
1930-01
1931-01
1932-01
1933-01
1934-01
1935-01
1936-01
1937-01
1938-01
1939-01
1940-01
1941-01
1942-01
1943-01
1944-01
1945-01
1946-01
1947-01
1948-01
1949-01
1950-01
1951-01
1952-01
1953-01
1954-01
1955-01
1956-01
1957-01
1958-01
1959-01
1960-01
1961-01
1962-01
1963-01
1964-01
1965-01
1966-01
1967-01
1968-01
1969-01
1970-01
1971-01
1972-01
1973-01
1974-01
1975-01
1976-01
1977-01
1978-01
1979-01
1980-01
1981-01
1982-01


In [ ]:
data_text = {}
for year_month in data:
    print(year_month)
    if len(data[year_month]) > 0 :
        texts = [] 
        ids = data[year_month]
        for i in ids:
            query = "http://api.larm.fm/v6/View/Get?view=Object&query={}&pageIndex=0&pageSize=20&sessionGUID=049da351-b81f-424e-82c4-1162926d3688&format=json&userHTTPStatusCodes=False".format(i)
            r = requests.get(query)
            metaxlm = r.json()['ModuleResults'][0]["Results"][0]["Metadatas"][0]["MetadataXml"]
            tree = ET.fromstring(metaxlm)
            txt = tree.getchildren()[2].text
            texts.append(txt)
        data_text[year_month] = texts

1950-01
1966-01
1961-01
1949-01
1938-01
1980-01
1926-01
1952-01
1964-01
1953-01
1958-01
1962-01
1933-01
1968-01
1972-01
1937-01
1935-01
1955-01
1925-01
1960-01
1978-01
1929-01
1941-01
1981-01
1944-01
1930-01
1945-01

In [ ]:
def clean(dat):
    wc = []
    words = dat.rstrip().split()
    for w in words:
        wc = regex.sub('',w)
        if len(wc)>1:
            if wc not in stop:
                data_ym.append(wc)
    return wc

In [ ]:
data_text_splitted = {}
for ym in data_text:
    print(ym)
    data_ym = []
    for dat in data_text[ym]:
        words = dat.rstrip().split()
        for w in words:
            wc = regex.sub('',w)
            if len(wc)>1:
                if wc not in stop:
                    data_ym.append(wc)
    data_text_splitted[ym] = Counter(data_ym).most_common(1000)

# csv

In [ ]:
f = open('myfile','w')
for ym in data_text:
    line = []
    line.append(ym)
    for word in  data_text_splitted[ym]:
        line.append(word[0])
        line.append(str(word[1]))
    l = " ".join(line)
    f.write(l+"\n")    

# json


In [ ]:
with open("results.json", "w") as f:
    json.dump(data_text_splitted_n, f)